In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pyomo.environ import *
from pyomo.opt import SolverFactory, TerminationCondition
from pyomo.contrib.appsi.solvers.gurobi import Gurobi as AppsiGurobi
from pyomo.core import TransformationFactory
from pyomo.opt import TerminationCondition as _TC
from collections import defaultdict
import logging
import networkx as nx
import time
import math
import sys

# Sample data
shapefileDemands = "small/smallHexGrid.shp"
shp_parking = "small/CharParkSmall.shp"
csv_distance_path = "small/shortestSmall.csv"
csv_spot = "ElPrice.csv"

# shapefileDemands = "full/demandHexGrid.shp"
# csv_distance_path = "full/shortestpath.csv"
# csv_spot = 'ElPrice.csv'
# shp_parking   = "full/CharParkfull.shp"

In [2]:
for h in list(logging.root.handlers): logging.root.removeHandler(h)

try:  sys.stdout.reconfigure(encoding="utf-8")  # no-op elsewhere
except Exception:  pass

fmt = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setFormatter(fmt)
stream_handler.setLevel(logging.DEBUG)
file_handler = logging.FileHandler('lbbd_optimization.log', mode='w', encoding='utf-8')
file_handler.setFormatter(fmt)
file_handler.setLevel(logging.INFO)
logging.basicConfig(level=logging.WARNING, handlers=[stream_handler, file_handler])

logging.getLogger('gurobipy').setLevel(logging.ERROR)
logging.getLogger('pyomo').setLevel(logging.WARNING)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO) 

# ---  termination checker--
def _is_optimal_tc(term) -> bool:
    # Prefer enum comparison when possible
    try:
        if term in (_TC.optimal, _TC.locallyOptimal, _TC.globallyOptimal):
            return True
    except Exception:
        pass
    # APPSI Results.termination_condition is also a TerminationCondition enum
    name = getattr(term, 'name', None)
    if isinstance(name, str):
        n = name.lower()
        return n in ('optimal', 'locallyoptimal', 'globallyoptimal')
    # Last resort: substring check
    s = str(term).lower()
    return ('optimal' in s) and ('infeasible' not in s)


# --- Lightweight logging helpers to understand master/SubP values ---
def _log_master_snapshot(master, it):
    try:
        tot_x = sum(float(value(master.x[i, c])) for i in master.I for c in master.C)
        tot_pv = sum(float(value(master.PV[i])) for i in master.I)
        tot_bt = sum(float(value(master.Batt[i])) for i in master.I)
        logger.info(f"[it {it:02d}] Master snapshot: sum_x={tot_x:.0f}  PV={tot_pv:.0f}  Batt={tot_bt:.0f}")
    except Exception:
        pass  # keep it safe in early iterations


# --- Compact per-slot logging  ---
LOG_SLOTS = False           # flip to True to re-enable
LP_OBJ_LOG_THRESHOLD = 5e4  # only log if LP/MIP obj is large

def _log_slot_snapshot(master, it, mon, t, Ri, Sj, mu_slot=None, lp_obj=None, mip_obj=None, thresh=5e4):
    # Log only when something is interesting
    if (lp_obj is None or lp_obj < thresh) and (mip_obj is None or mip_obj < thresh):
        return
    Rtot = sum(Ri.values())
    Stot = sum(Sj.values())
    parts = [f"[it {it:02d}] ({mon:>3s},{t:02d}) R={Rtot:.1f} S={Stot:.1f}"]
    if mu_slot:
        vals = list(mu_slot.values())
        if vals:
            vals_sorted = sorted(vals)
            med = vals_sorted[len(vals_sorted)//2]
            parts.append(f"mu[min/med/max]={min(vals):.3f}/{med:.3f}/{max(vals):.3f}")
    if lp_obj is not None:
        parts.append(f"LP={lp_obj:,.0f}")
    if mip_obj is not None:
        parts.append(f"MIP={mip_obj:,.0f}")
    logger.debug("  ".join(parts))


def _log_sp_top_flows(sp, k=5):
    try:
        if hasattr(sp, 'A') and len(list(sp.A)) > 0 and hasattr(sp, 'z'):
            flows = [((i, j), float(value(sp.z[i, j]))) for (i, j) in sp.A if value(sp.z[i, j]) > 1e-9]
            flows.sort(key=lambda x: -x[1])
            flows = flows[:k]
            if flows:
                logger.debug("      Top flows: " + ", ".join([f"{ij}:{v:.2f}" for (ij, v) in flows]))
    except Exception:
        pass

# Data preprocessing

In [3]:
def retail(p_ore):
    """Convert p_ore (öre/kWh) to SEK/kWh, then add grid‐fee, tax & VAT."""
    p_sek = p_ore / 100.0
    VAT, TAX, GRID, MARKUP = 0.25, 0.375, 0.45, 0.02
    return (p_sek + TAX + GRID + MARKUP) * (1 + VAT)

def get_distance_dict(csv_path):
    df = pd.read_csv(csv_path)
    df["distance_km"] = df["distance"] / 1000.0
    return {(int(r.from_HexID), int(r.to_HexID)): float(r.distance_km) for _, r in df.iterrows()}

def pvf_daily(r: float, n: int) -> float:
    return (r*(1+r)**n)/((1+r)**n - 1) / 365.0

# elegible Arc sets
def arc_active(i, j, t):
    """Return True when either end has public demand in slot t."""
    return (Demand_event[(i, t, 'public')] +
            Demand_event[(j, t, 'public')]) > 0.1

MONTHS = ['January','February','March','April','May','June',
          'July','August','September','October','November','December']
M_ABBR = {'January':'Jan','February':'Feb','March':'Mar','April':'Apr',
          'May':'May','June':'Jun','July':'Jul','August':'Aug',
          'September':'Sep','October':'Oct','November':'Nov','December':'Dec'}
N_MONTH = {'January':31,'February':28,'March':31,'April':30,'May':31,'June':30,
           'July':31,'August':31,'September':30,'October':31,'November':30,'December':31}
INTERVALS = list(range(1, 49))
DAYS = sum(N_MONTH.values())

# Home battery (per kWh usable)
BATT_CAPEX_KWH = 4200                  # SEK/kWh
BATT_CELL_CAP  = 10                    # kWh per model.Batt "cell"
BATT_LIFE      = 15
BATT_OM        = 0.01
C_RATE_KW = 11          # battery C-rate (kW)  
BP        = 0.8         # battery-protection factor
M_CH      = (C_RATE_KW * 0.5) * BP      # kWh that may flow in one 30-min slot
BATT_MAX   = 400
RHO = 2     # half-hour slots per hour
C_PUB = ["slow","medium","fast"]

# TOU Pricing
raw = pd.read_csv(csv_spot)
VAT_RATE, ELECTRICITY_TAX, GRID_FEE, RETAIL_MARKUP = 0.25, 0.375, 0.45, 0.02
tou = {m: {t: retail(raw.at[(t-1)//2, M_ABBR[m]]) for t in INTERVALS} for m in MONTHS}

# PV & irradiance (Gothenburg-specific)
# Average daily yield per kWp (kWh/kWp-day), PVGIS v5.3
# https://ec.europa.eu/jrc/en/PVGIS/tools/pvest (retrieved 2025-06-08)
monthly_kwh_per_kWp = {
    'January':   1.16,  # 
    'February':  2.47,  # 
    'March':     3.87,  # 
    'April':     5.27,  # 
    'May':       6.12,  # 
    'June':      6.42,  # 
    'July':      5.96,  # 
    'August':    5.05,  # 
    'September': 3.77,  # 
    'October':   2.33,  # 
    'November':  1.07,  # 
    'December':  0.67   # 
}

# intra-day shape 
base_hour_cf = np.array([
    0, 0, 0, 0, 0,
    0.02, 0.04, 0.06,
    0.10, 0.15, 0.20, 0.20, 0.20,
    0.16, 0.12, 0.08, 0.04,
    0, 0, 0, 0, 0, 0, 0
])
# expand to 48 half-hour slots
base_hh_cf = np.repeat(base_hour_cf, 2)

# normalizing factor: total kWh per kWp-day in base shape
E_base = base_hh_cf.sum() * 0.5  # 

# final monthly capacity‐factor table
pv_cf = { mon: {t: min((monthly_kwh_per_kWp[mon] / E_base) * base_hh_cf[t-1], 1.0) for t in INTERVALS}
         for mon in MONTHS}

#  Data Loading and Aggregation
logging.info("Loading hex grid and charger‑limits...")
gdf           = gpd.read_file(shapefileDemands)
chargerLimits = gpd.read_file(shp_parking)
gdf["HexID"]       = gdf["HexID"].astype(int)
gdf["charType"] = gdf["charType"].str.lower().str.strip()
HexIDs      = sorted(gdf["HexID"].unique())
time_indices = sorted(gdf["TimeIndex"].unique())

# Parking-lot & home-charger data
parking_gdf = gpd.read_file(shp_parking)
# parking_gdf['ParkingCap'] = (parking_gdf['ParkingCap']*0.95).astype(int)
parking_gdf['Panels'] = (parking_gdf['Panels']*0.5).astype(int)
parking_gdf["HexID"] = parking_gdf["HexID"].astype(int)
parking_gdf['ParkingCap'] += 17
parking_gdf['homeChar'] += 5

# parking-capacity upper bound for public chargers
orig_CL = parking_gdf.set_index("HexID")["ParkingCap"].to_dict()
CL = parking_gdf.set_index("HexID")["ParkingCap"].to_dict()

# exogenous number of home chargers already owned by residents
home_avail = parking_gdf.set_index("HexID")["homeChar"].to_dict()

# fill missing hexes with zeros
for i in HexIDs:
    CL.setdefault(i, 0)
    home_avail.setdefault(i, 0)

# Distances and allowed pairs
speed_car, speed_walk = 30.0, 6.0    # km/h
value_time            = 80.0         # SEK/h
x_kWh                 = 20.0          # energy per redirected trip (kWh)
redir_min             = 0.10          # lower bound we want to enforce (kWh)

beta = value_time * (1/speed_car + 2/speed_walk) 

# daily annuity function: capex (SEK), life (yr)
DISC = 0.06          # real annual discount rate r
ASSETS = {
    # name     CAPEX (SEK)   life (yr)   Ω = fixed O&M as frac. of CAPEX/yr- assume to be nil for now!
    'slow'  : (   8_500, 10, 0.00),
    'medium': (  16_500, 10, 0.00),
    'fast'  : ( 220_000, 10, 0.00),
    'PV'    : (   4_500, 25, 0.00),
    'Batt'  : (  40_000, 15, 0.00),
}

# compute SEK/day per charger
daily_cost = {}
for g, (capex, life, om) in ASSETS.items():
    cap = capex * pvf_daily(DISC, life)          # capital recovery /day
    o_m = capex * om / 365.0                     # fixed O&M /day
    daily_cost[g] = round(cap + o_m, 4)

charger_capacity = {"home": 3.5, "slow": 5.5, "medium": 11, "fast": 25}            # kWh/interval
charger_capacity_pub = {"slow": 5.5, "medium": 11, "fast": 25}
charger_price = {"slow": 5.50, "medium": 6.00, "fast": 6.50}         # CPO collects Charging tariffs (SEK per kWh)
eligible = {
    "home":   ["slow", "medium", "fast"],
    "public": ["slow", "medium", "fast"]
}

PRICE_MIN = min(charger_price[c] for c in ["slow", "medium", "fast"])
dist_dict = get_distance_dict(csv_distance_path)

# Average retail grid price (needed for SERV_COST)
avg_grid_price = np.mean([tou[m][t]              
                          for m in MONTHS
                          for t in INTERVALS])

MAX_DIST = 1.5        # km – redirection radius
allowed  = {(i, j) for (i, j), d in dist_dict.items() if i != j and d <= MAX_DIST}
logging.info(f"β (redirection cost per kWh·km) = {beta:.3f}")
T_TRIP = {(i, j): beta * dist_dict[(i, j)] for (i, j) in allowed}  # SEK/trip

# Demand aggregation  (work+public = public)
logging.info("Aggregating demand …")
grp = (gdf[["HexID","TimeIndex","Demand","charType"]]
       .groupby(["HexID","TimeIndex","charType"])["Demand"]
       .sum().reset_index())

D_home = {(i,t):0.0 for i in HexIDs for t in time_indices}
D_work = {(i,t):0.0 for i in HexIDs for t in time_indices}
D_pub  = {(i,t):0.0 for i in HexIDs for t in time_indices}

for _, r in grp.iterrows():
    i,t,et = int(r.HexID), int(r.TimeIndex), float(r.Demand)
    if r.charType=="home":     D_home[(i,t)]+=et
    elif r.charType=="work":   D_work[(i,t)]+=et
    else:                         D_pub[(i,t)]+=et

# maximum energy a resident can still take from own socket
HOME_KWH_PER_SLOT = charger_capacity["home"]        # = 3.5

# subtract the kWh that the fixed home-chargers could already supply
for i in HexIDs:
    cap_i = home_avail[i] * HOME_KWH_PER_SLOT
    for t in time_indices:
        D_home[(i, t)] = max(D_home[(i, t)] - cap_i, 0.0)

Demand_event = {}
for i in HexIDs:
    for t in time_indices:
        Demand_event[(i, t, 'home')]   = D_home[(i, t)]
        Demand_event[(i, t, 'public')] = D_work[(i, t)] + D_pub[(i, t)]

Demand_event_annual = {
    (i, m, t, e): Demand_event[(i, t, e)]        # same shape each Month
    for i in HexIDs
    for m in MONTHS
    for t in INTERVALS
    for e in ['home','public']
}

# Month-slot sparse set
ALLOWED_ST = {(i, j, m, t)                          # |I|² · |S| · |H|  -full
              for (i, j) in allowed
              for m in MONTHS
              for t in INTERVALS
              if arc_active(i, j, t)}

# prev-month lookup  (Jan-Dec wrap-around)
prev_month = {MONTHS[i]: MONTHS[i-1] if i > 0 else MONTHS[-1]
              for i in range(len(MONTHS))}

# PV Upper Bound Data
pv_shp = gpd.read_file(shp_parking)
pv_shp.rename(columns={"HexID": "HexID", "Panels": "PV_upper"}, inplace=True)
pv_shp["HexID"] = pv_shp["HexID"].astype(int)
pv_upper = pv_shp.set_index("HexID")["PV_upper"].to_dict()
for i in HexIDs:
    if i not in pv_upper: 
        pv_upper[i] = 0

# PV economics
LIFETIME_Y  = 25
CAPEX_W     = 16.0          #  SEK / W (after tax credit)
PANEL_W     = 0.40         #  kW   (400 W module)
capex_panel = CAPEX_W * PANEL_W * 1_000       # 400 W × 10 SEK/W  → 4 000 SEK
# pv_daily_annuity = round(capex_panel / (LIFETIME_Y*365), 4)  

# Panel can deliver 0.2 kWh in one 30-min slot if CF = 1
PV_KWH_PER_SLOT_AT_CF1 = PANEL_W * 0.5            # 0.4 kW * 0.5 h = 0.2 kWh

#  for parking limits
slack_ids = []
penalty_per_kWh  = 1000 # Penalty for unmet demand (SEK per kWh of slack)
# penalty dominates price so "not serving" is never profitable
assert penalty_per_kWh >= 1.1 * max(charger_price.values())
PRICE_REF = charger_price['medium']   # 6.00 in your data
#  undirected graph whose nodes are your hex‐cells
G = nx.Graph()
G.add_nodes_from(HexIDs)
# Connect any two cells that are within redirection radius
G.add_edges_from(allowed)   # allowed = {(i,j) …}

#  total public demand per cell
pub_demand = {
    i: sum(Demand_event[i, t, 'public'] for t in time_indices)
    for i in HexIDs
}

# For each connected component that has public demand but zero parking slots
for comp in nx.connected_components(G):
    if (any(pub_demand[i] > 0 for i in comp)
     and all(CL[i] == 0     for i in comp)):
        # pick the busiest node in that component
        i_star = max(comp, key=lambda i: pub_demand[i])
        CL[i_star] = 2   # at least one charger‐slot there

# charger_capacity["slow"] is 5.5 kWh per interval
slow_cap = charger_capacity['slow']
for i in slack_ids:
    # what’s its peak public‐demand in any half-hour?
    peak = max(Demand_event[(i, t, 'public')] for t in time_indices)
    # how many slow‐chargers to fully cover that?
    needed = math.ceil(peak / slow_cap)
    CL[i] = needed

# ------------------------------
# Summary logs
# ------------------------------
def print_data_summary():
    slot_totals_repday = np.array([sum(Demand_event[(i, t, 'public')] for i in HexIDs) for t in INTERVALS])
    peak_t = int(slot_totals_repday.argmax() + 1)
    annual_pub_demand = sum(N_MONTH[m] * slot_totals_repday.sum() for m in MONTHS)
    cell_repday = {i: sum(Demand_event[(i, t, 'public')] for t in INTERVALS) for i in HexIDs}
    i_peak = max(cell_repday, key=cell_repday.get)
    
    print(f"\n DEMAND STATS")
    print("-" * 50)
    print(f"Cells: {len(HexIDs)} | Months: {len(MONTHS)} | Time intervals: {len(INTERVALS)}")
    print(f"Representative day public demand: {slot_totals_repday.sum():,.1f} kWh")
    print(f"Peak demand: {slot_totals_repday[peak_t-1]:,.1f} kWh at interval {peak_t}")
    print(f"95th percentile demand: {np.quantile(slot_totals_repday, 0.95):,.1f} kWh")
    print(f"Annual public demand: {annual_pub_demand:,.0f} kWh/yr")
    print(f"Peak demand cell: HexID {i_peak} with {cell_repday[i_peak]:,.1f} kWh/day")
    
    # Price STATS
    price_vec = np.array([tou[m][t] for m in MONTHS for t in INTERVALS])
    print(f"\n PRICE STATS")
    print("-" * 50)
    print(f"Retail ToU price (SEK/kWh):")
    print(f"  Mean: {price_vec.mean():.3f} | Median: {np.median(price_vec):.3f}")
    print(f"  Min: {price_vec.min():.3f} | Max: {price_vec.max():.3f}")
    print(f"Public charging tariffs:")
    for c in ["slow","medium","fast"]:
        print(f"  {c:6}: {charger_price[c]:.2f} SEK/kWh")
    
    # Margin analysis
    print(f"\n MARGIN ANALYSIS (Price - ToU)")
    print("-" * 50)
    for c in ["slow","medium","fast"]:
        mvec = np.maximum(0.0, np.array([charger_price[c] - tou[m][t] for m in MONTHS for t in INTERVALS]))
        print(f"  {c:6}: Mean={mvec.mean():.3f} | Max={mvec.max():.3f} SEK/kWh")
    
    # Investment costs
    print(f"\n INVESTMENT COSTS")
    print("-" * 50)
    print("Daily annuity (SEK/day):")
    for k in ['slow','medium','fast','PV','Batt']:
        print(f"  {k:6}: {daily_cost[k]:6.2f}  →  Annual: {DAYS*daily_cost[k]:>8,.0f} SEK/yr")
    
    # Infrastructure limits
    pv_cells = sum(1 for i in HexIDs if pv_upper.get(i, 0) > 0)
    pv_total_ub = int(sum(pv_upper.get(i, 0) for i in HexIDs))
    print(f"\n  INFRASTRUCTURE LIMITS")
    print("-" * 50)
    print(f"PV panels: {pv_cells} cells with capacity | Total UB: {pv_total_ub:,} panels")
    print(f"Battery: 10 kWh cells | Max per cell: {BATT_MAX} | Charge limit: {M_CH:.1f} kWh/30min")
    print(f"PV generation: {PV_KWH_PER_SLOT_AT_CF1:.2f} kWh/panel/slot at CF=1")
    
    # Redirection network
    dist_allowed = [dist_dict[p] for p in allowed]
    T_allowed = [T_TRIP[p] for p in allowed] if len(allowed) else [0.0]
    print(f"\n REDIRECTION NETWORK")
    print("-" * 50)
    print(f"Radius: {MAX_DIST} km | Neighbor pairs: {len(allowed)}")
    print(f"Active arc-slots: {len(ALLOWED_ST):,}")
    if dist_allowed:
        print(f"Distances: {min(dist_allowed):.2f}-{max(dist_allowed):.2f} km")
        print(f"  Median: {np.median(dist_allowed):.2f} km | 95th: {np.quantile(dist_allowed,0.95):.2f} km")
        print(f"Incentive cost: {min(T_allowed):.1f}-{max(T_allowed):.1f} SEK/kWh")
        print(f"  Trip quantum: {x_kWh:.0f} kWh | Min flow: {redir_min:.2f} kWh")
    
    # Home chargers
    home_total = int(sum(home_avail.values()))
    print(f"\n HOME CHARGERS")
    print("-" * 50)
    print(f"Total units: {home_total:,}")
    print(f"Capacity: {HOME_KWH_PER_SLOT:.1f} kWh/slot per charger")
    
    # Parking capacity adjustments
    try:
        num_adj = sum(1 for i in HexIDs if CL[i] != orig_CL[i])
        if num_adj > 0:
            print(f"\n PARKING ADJUSTMENTS")
            print("-" * 50)
            print(f"Adjusted {num_adj} cells to ensure feasibility")
    except Exception:
        pass

print_data_summary()


 DEMAND STATS
--------------------------------------------------
Cells: 63 | Months: 12 | Time intervals: 48
Representative day public demand: 19,356.1 kWh
Peak demand: 1,043.2 kWh at interval 19
95th percentile demand: 966.6 kWh
Annual public demand: 7,064,979 kWh/yr
Peak demand cell: HexID 36 with 1,669.0 kWh/day

 PRICE STATS
--------------------------------------------------
Retail ToU price (SEK/kWh):
  Mean: 1.679 | Median: 1.571
  Min: 1.001 | Max: 3.340
Public charging tariffs:
  slow  : 5.50 SEK/kWh
  medium: 6.00 SEK/kWh
  fast  : 6.50 SEK/kWh

 MARGIN ANALYSIS (Price - ToU)
--------------------------------------------------
  slow  : Mean=3.821 | Max=4.499 SEK/kWh
  medium: Mean=4.321 | Max=4.999 SEK/kWh
  fast  : Mean=4.821 | Max=5.499 SEK/kWh

 INVESTMENT COSTS
--------------------------------------------------
Daily annuity (SEK/day):
  slow  :   3.16  →  Annual:    1,155 SEK/yr
  medium:   6.14  →  Annual:    2,242 SEK/yr
  fast  :  81.89  →  Annual:   29,891 SEK/yr
  P

# Master Problem

In [4]:
def create_master_problem():
    margin_max = {(mon, t): max(0.0, max(charger_price[c] - tou[mon][t] for c in C_PUB))
                  for mon in MONTHS for t in INTERVALS}

    model = ConcreteModel()
    model.I = Set(initialize=HexIDs)
    model.M = Set(initialize=MONTHS)
    model.H = Set(initialize=INTERVALS)
    model.C = Set(initialize=C_PUB)
    model.B = Set(initialize=['home','public'])

    model.Demand = Param(model.I, model.M, model.H, model.B, initialize=Demand_event_annual, within=NonNegativeReals)
    model.K = Param(model.C, initialize=charger_capacity_pub)
    model.Price = Param(model.C, initialize=charger_price)
    model.CL = Param(model.I, initialize=CL)
    model.tou = Param(model.M, model.H, initialize=lambda m, mon, t: tou[mon][t])
    model.PV_panel_cap = Param(initialize=PV_KWH_PER_SLOT_AT_CF1)
    model.p_pv = Param(model.M, model.H, initialize=lambda m, mon, t: pv_cf[mon][t])
    model.PV_upper = Param(model.I, initialize=pv_upper)
    model.PVF_cost = Param(model.C, initialize=lambda m, c: daily_cost[c])
    model.PVF_PV = Param(initialize=daily_cost['PV'])
    model.PVF_Batt = Param(initialize=daily_cost['Batt'])
    model.Batt_cell_cap = Param(initialize=10)
    model.eta_ch = Param(initialize=0.95)
    model.eta_dis = Param(initialize=0.95)
    model.prev_mon = Param(model.M, initialize=prev_month, within=Any)
    model.prev = Param(model.H, initialize={t: (t-1 if t > 1 else 48) for t in INTERVALS}, within=PositiveIntegers)

    def xbnds(m, i, c): return (0, CL[i])
    model.x = Var(model.I, model.C, domain=NonNegativeIntegers, bounds=xbnds)
    model.PV = Var(model.I, domain=NonNegativeIntegers)
    model.Batt = Var(model.I, domain=NonNegativeIntegers)

    model.edisp = Var(model.I, model.M, model.H, model.C, model.B, domain=NonNegativeReals)
    model.grid_dir = Var(model.I, model.M, model.H, domain=NonNegativeReals)
    model.grid_batt = Var(model.I, model.M, model.H, domain=NonNegativeReals)
    model.pv_dir = Var(model.I, model.M, model.H, domain=NonNegativeReals)
    model.pv_batt = Var(model.I, model.M, model.H, domain=NonNegativeReals)
    model.batt_discharge = Var(model.I, model.M, model.H, domain=NonNegativeReals)
    model.soc = Var(model.I, model.M, model.H, domain=NonNegativeReals)
    model.delta = Var(model.I, model.M, model.H, domain=Binary)
    model.slack = Var(model.I, model.M, model.H, model.B, domain=NonNegativeReals)

    model.R = Var(model.I, model.M, model.H, domain=NonNegativeReals)
    model.S = Var(model.I, model.M, model.H, domain=NonNegativeReals)
    model.ThetaRedir = Var(model.M, model.H, domain=NonNegativeReals)

    def pub_limit_rule(m, i):
        return sum(m.x[i, c] for c in m.C) <= m.CL[i]
    model.PublicLimit = Constraint(model.I, rule=pub_limit_rule)

    def site_capacity_rule(m, i, mon, t):
        return sum(m.edisp[i, mon, t, c, b] for b in m.B for c in m.C) <= sum(m.K[c] * m.x[i, c] for c in m.C)
    model.SiteCap = Constraint(model.I, model.M, model.H, rule=site_capacity_rule)

    def demand_home_rule(m, i, mon, t):
        return sum(m.edisp[i, mon, t, c, 'home'] for c in m.C) + m.slack[i, mon, t, 'home'] == m.Demand[i, mon, t, 'home']
    model.DemandHome = Constraint(model.I, model.M, model.H, rule=demand_home_rule)

    def demand_public_rule(m, i, mon, t):
        return sum(m.edisp[i, mon, t, c, 'public'] for c in m.C) + m.R[i, mon, t] + m.slack[i, mon, t, 'public'] == m.Demand[i, mon, t, 'public']
    model.DemandPublic = Constraint(model.I, model.M, model.H, rule=demand_public_rule)

    def spare_capacity_rule(m, i, mon, t):
        return m.S[i, mon, t] + sum(m.edisp[i, mon, t, c, b] for b in m.B for c in m.C) == sum(m.K[c] * m.x[i, c] for c in m.C)
    model.SpareCapacity = Constraint(model.I, model.M, model.H, rule=spare_capacity_rule)

    def energy_balance_rule(m, i, mon, t):
        supply = m.grid_dir[i, mon, t] + m.pv_dir[i, mon, t] + m.batt_discharge[i, mon, t]
        demand = sum(m.edisp[i, mon, t, c, b] for b in m.B for c in m.C)
        return supply == demand
    model.EnergyBalance = Constraint(model.I, model.M, model.H, rule=energy_balance_rule)

    def pv_generation_rule(m, i, mon, t):
        return m.pv_dir[i, mon, t] + m.pv_batt[i, mon, t] <= m.PV_panel_cap * m.PV[i] * m.p_pv[mon, t]
    model.PVGeneration = Constraint(model.I, model.M, model.H, rule=pv_generation_rule)

    LAST_H = max(INTERVALS)

    def battery_dynamics_rule(m, i, mon, t):
        if (mon == 'January') and (t == 1):
            return m.soc[i, mon, t] == 0.20 * m.Batt_cell_cap * m.Batt[i]
        if t == 1:
            pm = m.prev_mon[mon]
            return m.soc[i, mon, t] == m.soc[i, pm, LAST_H]
        prev_soc = m.soc[i, mon, m.prev[t]]
        charge = m.eta_ch * (m.grid_batt[i, mon, t] + m.pv_batt[i, mon, t])
        discharge = (1.0 / m.eta_dis) * m.batt_discharge[i, mon, t]
        return m.soc[i, mon, t] == prev_soc + charge - discharge
    model.BatteryDynamics = Constraint(model.I, model.M, model.H, rule=battery_dynamics_rule)

    def battery_discharge_limit_rule(m, i, mon, t):
        return m.batt_discharge[i, mon, t] <= m.soc[i, mon, t]
    model.BatteryDischargeLimit = Constraint(model.I, model.M, model.H, rule=battery_discharge_limit_rule)

    def batt_cap_rule(m, i):
        return m.Batt[i] <= BATT_MAX
    model.BattCap = Constraint(model.I, rule=batt_cap_rule)

    def pv_upper_bound_rule(m, i):
        return m.PV[i] <= m.PV_upper[i]
    model.PVUpperBound = Constraint(model.I, rule=pv_upper_bound_rule)

    def charge_only(m, i, mon, t):
        return m.grid_batt[i, mon, t] + m.pv_batt[i, mon, t] <= M_CH * m.delta[i, mon, t]
    model.ChargeOnly = Constraint(model.I, model.M, model.H, rule=charge_only)

    def discharge_only(m, i, mon, t):
        return m.batt_discharge[i, mon, t] <= M_CH * (1 - m.delta[i, mon, t])
    model.DischargeOnly = Constraint(model.I, model.M, model.H, rule=discharge_only)

    def battery_capacity_upper(m, i, mon, t):
        return m.soc[i, mon, t] <= 0.9 * m.Batt_cell_cap * m.Batt[i]
    model.BatteryCapUB = Constraint(model.I, model.M, model.H, rule=battery_capacity_upper)

    def battery_capacity_lower(m, i, mon, t):
        return m.soc[i, mon, t] >= 0.1 * m.Batt_cell_cap * m.Batt[i]
    model.BatteryCapLB = Constraint(model.I, model.M, model.H, rule=battery_capacity_lower)

    def annual_soc_closure_rule(m, i):
        return m.soc[i, 'December', max(m.H)] == m.soc[i, 'January', 1]
    model.SOCclosure = Constraint(model.I, rule=annual_soc_closure_rule)

    def theta_init_bound(m, mon, t):
        # Safe finite cap: price × (annualized) public energy that even could be redirected
        return m.ThetaRedir[mon, t] <= N_MONTH[mon] * PRICE_REF * sum(m.Demand[i, mon, t, 'public'] for i in m.I)
    model.InitThetaUB = Constraint(model.M, model.H, rule=theta_init_bound)

    model.benders_cuts = ConstraintList()

    rev_local = sum(N_MONTH[mon] * model.Price[c] * model.edisp[i, mon, t, c, b]
                    for i in model.I for mon in model.M for t in model.H for b in model.B for c in model.C)
    cost_grid = (sum(N_MONTH[mon] * model.tou[mon, t] * model.grid_dir[i, mon, t]
                     for i in model.I for mon in model.M for t in model.H)
                 + sum(N_MONTH[mon] * model.tou[mon, t] * model.grid_batt[i, mon, t]
                     for i in model.I for mon in model.M for t in model.H))
    cost_slack = penalty_per_kWh * sum(N_MONTH[mon] * model.slack[i, mon, t, b]
                                       for i in model.I for mon in model.M for t in model.H for b in model.B)
    capex = DAYS * (sum(model.PVF_cost[c] * model.x[i, c] for i in model.I for c in model.C)
                    + sum(model.PVF_PV * model.PV[i] for i in model.I)
                    + sum(model.PVF_Batt * model.Batt[i] for i in model.I))
    redir_value = sum(model.ThetaRedir[mon, t] for mon in model.M for t in model.H)

    model.obj = Objective(expr=rev_local + redir_value - cost_grid - cost_slack - capex, sense=maximize)
    return model

In [5]:
def active_pairs_for_slot(mon, t):
    return [(i, j) for (i, j, mm, tt) in ALLOWED_ST if mm == mon and tt == t]

def build_RS_from_master(master, mon, t, tol=1e-9):
    R = {i: float(value(master.R[i, mon, t])) for i in master.I if value(master.R[i, mon, t]) > tol}
    S = {i: float(value(master.S[i, mon, t])) for i in master.I if value(master.S[i, mon, t]) > tol}
    return R, S

def extract_transport_duals(sp):
    """Return duals for Supply[i] and Capacity[j], safely handling missing/Skipped constraints."""
    alpha, beta = {}, {}

    has_supply   = hasattr(sp, 'Supply')
    has_capacity = hasattr(sp, 'Capacity')

    if has_supply:
        for i in list(sp.I):
            if i in sp.Supply:                 # only if this ConstraintData exists (not skipped)
                cd = sp.Supply[i]
                try:
                    alpha[i] = float(sp.dual[cd])
                except KeyError:
                    alpha[i] = 0.0
            else:
                alpha[i] = 0.0

    if has_capacity:
        for j in list(sp.J):
            if j in sp.Capacity:
                cd = sp.Capacity[j]
                try:
                    beta[j] = float(sp.dual[cd])
                except KeyError:
                    beta[j] = 0.0
            else:
                beta[j] = 0.0

    return alpha, beta


def add_transport_cut(master, mon, t, alpha, beta):
    # Dual objective already equals the (scaled) primal objective of spLP by strong duality.
    expr = master.ThetaRedir[mon, t] <= (
        sum(alpha.get(i, 0.0) * master.R[i, mon, t] for i in master.I) +
        sum(beta.get(j, 0.0) * master.S[j, mon, t] for j in master.I)
    )
    return master.benders_cuts.add(expr)

# --- price mix under any fixed design model (master clone / LP clone) ---
def _price_bar_by_cell_from_design(m_model):
    """Compute average SEK/kWh across installed charger mix for each destination j."""
    pbar_local = {}
    for j in HexIDs:
        denom = sum(charger_capacity_pub[c] * int(round(value(m_model.x[j, c]))) for c in C_PUB)
        if denom > 1e-9:
            num = sum(charger_price[c] * charger_capacity_pub[c] * int(round(value(m_model.x[j, c]))) for c in C_PUB)
            pbar_local[j] = num / denom
        else:
            # No bays: use the highest public price as a safe upper bound
            pbar_local[j] = max(charger_price.values())
    return pbar_local


# --- integer (logic-based) optimality cut for θ ---
def add_integer_optimality_cut(master, mon, t, theta_star, Ri, Sj, mu_slot, price_dict):
    """
    Global linear upper bound for the true integer recourse value:
        ThetaRedir[mon,t] <= theta_star
                              + sum_i L_i * (R[i,mon,t] - Ri[i])
                              + sum_j U_j * (S[j,mon,t] - Sj[j])
    with nonnegative per-kWh slopes L_i, U_j derived from current μ and prices.
    """
    # Per-kWh margin for arc (i,j) at current μ and prices
    def _margin_ij(i, j):
        return max(0.0, price_dict[j] - mu_slot[j] - T_TRIP.get((i, j), float('inf'))/x_kWh)

    Nm = N_MONTH[mon]
    Ikeys = list(Ri.keys())
    Jkeys = list(Sj.keys())

    # L_i = best (nonnegative) per-kWh margin reachable from i; U_j = best into j
    A_f = set(active_pairs_for_slot(mon, t))  # only arcs usable in this (mon,t)
    L = {i: Nm * max((_margin_ij(i, j) for j in Jkeys if (i, j) in A_f), default=0.0) for i in Ikeys}
    U = {j: Nm * max((_margin_ij(i, j) for i in Ikeys if (i, j) in A_f), default=0.0) for j in Jkeys}

    expr = (master.ThetaRedir[mon, t] <= theta_star
            + sum(L[i] * (master.R[i, mon, t] - Ri[i]) for i in Ikeys)
            + sum(U[j] * (master.S[j, mon, t] - Sj[j]) for j in Jkeys))
    return master.benders_cuts.add(expr)


def extract_mu_from_master_lp(master, x_sol, pv_sol, batt_sol, delta_sol, optLP):
    """
    Clone the master, fix integer/binary investments to incumbent, solve LP relaxation,
    and return (objective_value, mu_dict) where
      mu_dict[(j, mon, t)] = dual on EnergyBalance[j, mon, t].
    """
    # 1) Clone & fix ints/binaries
    m_lp = master.clone()

    # Fix public-charger counts
    for (i, c), v in x_sol.items():
        m_lp.x[i, c].fix(int(round(v)))
    # Fix PV/Battery counts
    for i, v in pv_sol.items():
        m_lp.PV[i].fix(int(round(v)))
    for i, v in batt_sol.items():
        m_lp.Batt[i].fix(int(round(v)))
    # Fix binary charge/discharge indicator
    for (i, mon, t), v in delta_sol.items():
        m_lp.delta[i, mon, t].fix(int(round(v)))

    # 2) Attach a dual suffix, solve LP with simplex and no dual reductions
    if not hasattr(m_lp, 'dual'):
        m_lp.dual = Suffix(direction=Suffix.IMPORT)

    # IMPORTANT: use a non-persistent solver for duals on the clone
    res = optLP.solve(m_lp)  # options already set: Method=1, DualReductions=0
    term = getattr(res, 'termination_condition', getattr(res.solver, 'termination_condition', None))
    if not _is_optimal_tc(term):
        raise RuntimeError(f"Master LP for dual extraction not optimal: {term}")


    # 3) Harvest μ for each EnergyBalance constraint
    mu = {(j, mon, t): float(m_lp.dual[m_lp.EnergyBalance[j, mon, t]])
          for j in m_lp.I for mon in m_lp.M for t in m_lp.H}

    return float(value(m_lp.obj)), mu

# Subproblems

In [6]:
def create_subproblem_lp(mon, t, R_dict, S_dict, mu_dict, price_dict, arcs=None):
    I = sorted(R_dict.keys())
    J = sorted(S_dict.keys())
    if len(I) == 0 or len(J) == 0:
        m = ConcreteModel()
        m.obj = Objective(expr=0.0, sense=maximize)
        m.dual = Suffix(direction=Suffix.IMPORT)
        m.I = Set(initialize=[])
        m.J = Set(initialize=[])
        return m
    A = arcs if arcs is not None else active_pairs_for_slot(mon, t)
    A_f = [(i, j) for (i, j) in A if i in I and j in J]
    if not A_f:
        m = ConcreteModel()
        m.I = Set(initialize=I)
        m.J = Set(initialize=J)
        m.A = Set(dimen=2, initialize=[])          # present but empty
        m.R = Param(m.I, initialize={i: R_dict[i] for i in I})
        m.S = Param(m.J, initialize={j: S_dict[j] for j in J})
        m.z = Var(m.A, domain=NonNegativeReals)     # present but no vars
        m.obj = Objective(expr=0.0, sense=maximize)
        m.dual = Suffix(direction=Suffix.IMPORT)
        return m

    coef = {(i, j): N_MONTH[mon] * (price_dict[j] - mu_dict[j] - T_TRIP[(i, j)]/x_kWh) for (i, j) in A_f}

    m = ConcreteModel()
    m.I = Set(initialize=I)
    m.J = Set(initialize=J)
    m.A = Set(dimen=2, initialize=A_f)
    m.R = Param(m.I, initialize={i: R_dict[i] for i in I})
    m.S = Param(m.J, initialize={j: S_dict[j] for j in J})
    m.coef = Param(m.A, initialize=coef)
    m.z = Var(m.A, domain=NonNegativeReals)
    
    def _sup(m, i):
        terms = [m.z[i, j] for j in m.J if (i, j) in m.A]
        if not terms:
            return Constraint.Skip  # no outgoing arcs from i in this slot
        return sum(terms) <= m.R[i]
    
    def _cap(m, j):
        terms = [m.z[i, j] for i in m.I if (i, j) in m.A]
        if not terms:
            return Constraint.Skip  # no incoming arcs to j in this slot
        return sum(terms) <= m.S[j]
    
    m.Supply = Constraint(m.I, rule=_sup)
    m.Capacity = Constraint(m.J, rule=_cap)

    m.obj = Objective(expr=sum(m.coef[i, j] * m.z[i, j] for (i, j) in m.A), sense=maximize)
    m.dual = Suffix(direction=Suffix.IMPORT)
    return m


def create_subproblem_mip(mon, t, R_dict, S_dict, mu_dict, price_dict, arcs=None, enforce_min_quantum=False):
    I = sorted(R_dict.keys())
    J = sorted(S_dict.keys())
    if len(I) == 0 or len(J) == 0:
        m = ConcreteModel()
        m.obj = Objective(expr=0.0, sense=maximize)
        return m
    A = arcs if arcs is not None else active_pairs_for_slot(mon, t)
    A_f = [(i, j) for (i, j) in A if i in I and j in J]
    if not A_f:
        m = ConcreteModel()
        m.I = Set(initialize=I)
        m.J = Set(initialize=J)
        m.A = Set(dimen=2, initialize=[])
        m.R = Param(m.I, initialize={i: R_dict[i] for i in I})
        m.S = Param(m.J, initialize={j: S_dict[j] for j in J})
        m.z = Var(m.A, domain=NonNegativeReals)
        m.ntrip = Var(m.A, domain=NonNegativeIntegers)
        m.obj = Objective(expr=0.0, sense=maximize)
        return m

    rev_coef  = {(i, j): N_MONTH[mon] * (price_dict[j] - mu_dict[j]) for (i, j) in A_f}
    trip_cost = {(i, j): N_MONTH[mon] * T_TRIP[(i, j)] for (i, j) in A_f}

    m = ConcreteModel()
    m.I = Set(initialize=I)
    m.J = Set(initialize=J)
    m.A = Set(dimen=2, initialize=A_f)
    m.R = Param(m.I, initialize={i: R_dict[i] for i in I})
    m.S = Param(m.J, initialize={j: S_dict[j] for j in J})
    m.z = Var(m.A, domain=NonNegativeReals)
    m.ntrip = Var(m.A, domain=NonNegativeIntegers)
    if enforce_min_quantum: m.Y = Var(m.A, domain=Binary)

    def _sup(m, i):
        terms = [m.z[i, j] for j in m.J if (i, j) in m.A]
        if not terms:
            return Constraint.Skip
        return sum(terms) <= m.R[i]
    
    def _cap(m, j):
        terms = [m.z[i, j] for i in m.I if (i, j) in m.A]
        if not terms:
            return Constraint.Skip
        return sum(terms) <= m.S[j]
    
    m.Supply = Constraint(m.I, rule=_sup)
    m.Capacity = Constraint(m.J, rule=_cap)

    def _link(m, i, j): return m.z[i, j] <= x_kWh * m.ntrip[i, j]
    m.Link = Constraint(m.A, rule=_link)
    if enforce_min_quantum:
        def _minq(m, i, j): return m.z[i, j] >= redir_min * m.Y[i, j]
        def _trip_on(m, i, j): return m.ntrip[i, j] >= m.Y[i, j]
        m.MinQuantum = Constraint(m.A, rule=_minq)
        m.TripOn = Constraint(m.A, rule=_trip_on)
    m.obj = Objective(expr=sum(rev_coef[i, j] * m.z[i, j] - trip_cost[i, j] * m.ntrip[i, j] for (i, j) in m.A),
                      sense=maximize)
    return m

# LBBD loop for optimal solution convergence

In [7]:
class LogicBasedBenders:
    def __init__(self, tolerance=0.001, max_iterations=20, eval_mip_every=2, submip_timelimit=None,
                 debug_first_k_slots=3, stop_if_no_cuts=True, no_progress_patience=2, rel_improve_tol=1e-6):
        self.tolerance = float(tolerance)
        self.max_iterations = int(max_iterations)
        self.eval_mip_every = int(eval_mip_every)
        self.submip_timelimit = submip_timelimit
        self.iteration_history = []
        self.debug_first_k_slots = 0
        
        # cache: best (lowest) RHS seen so far for ThetaRedir[mon,t]
        self.best_theta_rhs = defaultdict(lambda: float('inf'))
        self.core_R = { (mon,t): defaultdict(float) for mon in MONTHS for t in INTERVALS }
        self.core_S = { (mon,t): defaultdict(float) for mon in MONTHS for t in INTERVALS }
        self.core_weight = defaultdict(float)  # key=(mon,t)
        # early-stop logic
        self.stop_if_no_cuts = bool(stop_if_no_cuts)
        self.no_progress_patience = int(no_progress_patience)
        self.rel_improve_tol = float(rel_improve_tol)
        self._prev_metrics = {'UB': None, 'LB': None, 'gap': None}
        self._stall_count = 0

    def _update_core_point(self, mon, t, Ri, Sj, w=0.2):
        key = (mon, t)
        self.core_weight[key] = self.core_weight.get(key, 0.0) * (1.0 - w) + w
        cw = self.core_weight[key]
        for i, v in Ri.items():
            self.core_R[key][i] = (1.0 - w) * self.core_R[key].get(i, 0.0) + w * v
        for j, v in Sj.items():
            self.core_S[key][j] = (1.0 - w) * self.core_S[key].get(j, 0.0) + w * v
    
    def _mw_rescale(self, mon, t, alpha, beta):
        # Magnanti–Wong style normalization at the core point (cheap variant)
        key = (mon, t)
        num = 0.0
        for i, a in alpha.items(): num += a * self.core_R[key].get(i, 0.0)
        for j, b in beta.items():  num += b * self.core_S[key].get(j, 0.0)
        if num > 1e-9:
            alpha = {i: a/num for i, a in alpha.items()}
            beta  = {j: b/num for j, b in beta.items()}
        return alpha, beta

    def _configure_master_solver(self):
        opt = AppsiGurobi()
        # quiet logs
        opt.config.stream_solver = False
        # set common Gurobi opts (APPSI exposes options via .options; fall back to .solver_options if needed)
        for k, v in {'Threads': 8, 'MIPGap': 1e-6, 'Presolve': 2, 'Cuts': 2, 'MIPFocus': 1, 'OutputFlag': 0}.items():
            try:
                opt.options[k] = v
            except Exception:
                try:
                    opt.solver_options[k] = v
                except Exception:
                    pass
        return opt

    def _configure_lp_solver(self):
        opt = SolverFactory('gurobi')
        # simplex + no dual reductions -> stable duals for cuts
        opt.options.update({
            'Method': 1,           # dual simplex is also fine; we use primal simplex for stability
            'DualReductions': 0,   # required to keep meaningful duals
            'Presolve': 0,         # ensure constraints aren't eliminated
            'InfUnbdInfo': 1,      # extra info on infeas/unbounded (safer diagnostics)
            'OutputFlag': 0
        })
        return opt

    def _configure_mip_solver(self):
        opt = SolverFactory('gurobi')
        opt.options.update({'OutputFlag': 0, 'MIPGap': 1e-6, 'Threads': 8, 'Presolve': 2})
        if self.submip_timelimit is not None:
            opt.options['TimeLimit'] = int(self.submip_timelimit)
        return opt

    def solve(self):
        master = create_master_problem()
        optM = self._configure_master_solver()
        optLP = self._configure_lp_solver()
        optSM = self._configure_mip_solver()

        best_UB = float('inf')
        best_LB = float('-inf')
        best_x = None
        best_PV = None
        best_Batt = None

        for it in range(1, self.max_iterations + 1):
            slot_summ = []  # clear per iteration
            resM = optM.solve(master)
            termM = getattr(resM, 'termination_condition', getattr(resM, 'solver', None) and resM.solver.termination_condition)
            if not _is_optimal_tc(termM):
                raise RuntimeError(f"Master not optimal (status={termM})")

            UB_cand = float(value(master.obj))
            best_UB = min(best_UB, UB_cand)
            _log_master_snapshot(master, it)

            x_sol = {(i, c): value(master.x[i, c]) for i in master.I for c in master.C}
            pv_sol = {i: value(master.PV[i]) for i in master.I}
            batt_sol = {i: value(master.Batt[i]) for i in master.I}
            delta_sol = {(i, mon, t): value(master.delta[i, mon, t]) for i in master.I for mon in master.M for t in master.H}

            # Compute price mix (SEK/kWh) seen by redirected energy at each destination cell j
            def _price_bar_by_cell(x_sol_local):
                pbar_local = {}
                for j in HexIDs:
                    denom = sum(charger_capacity_pub[c] * x_sol_local.get((j, c), 0.0) for c in C_PUB)
                    if denom > 1e-9:
                        num = sum(charger_price[c] * charger_capacity_pub[c] * x_sol_local.get((j, c), 0.0) for c in C_PUB)
                        pbar_local[j] = num / denom
                    else:
                        pbar_local[j] = max(charger_price.values())  # safe upper price if no bays yet
                return pbar_local

            pbar = _price_bar_by_cell(x_sol)

            # LP relaxation to get μ for EnergyBalance (dual multipliers)
            opLP_val, mu = extract_mu_from_master_lp(master, x_sol, pv_sol, batt_sol, delta_sol, optLP)
            logger.debug(f"[it {it:02d}] Master-LP objective for duals: {opLP_val:,.2f}")

            # Simple box trust-region (optional): tighten bounds around current design
            TR = 10  # allow +-10 units change per asset per cell; tune if needed
            for (i, c), v in x_sol.items():
                lb = max(0, int(round(v)) - TR)
                ub = min(int(round(v)) + TR, CL[i])
                master.x[i, c].setlb(lb); master.x[i, c].setub(ub)
            for i, v in pv_sol.items():
                lb = max(0, int(round(v)) - TR); ub = int(round(v)) + TR
                master.PV[i].setlb(lb); master.PV[i].setub(ub)
            for i, v in batt_sol.items():
                lb = max(0, int(round(v)) - TR); ub = min(BATT_MAX, int(round(v)) + TR)
                master.Batt[i].setlb(lb); master.Batt[i].setub(ub)

            added_cuts = 0
            theta_mip_total = 0.0
            theta_heur_total = 0.0  # NEW: fast rounding-based feasible θ
            debug_slots_printed = 0

            for mon in MONTHS:
                for t in INTERVALS:
                    # supply/demand for redirection at this slot
                    Ri = {i: float(value(master.R[i, mon, t])) for i in master.I if value(master.R[i, mon, t]) > 1e-9}
                    Sj = {j: float(value(master.S[j, mon, t])) for j in master.I if value(master.S[j, mon, t]) > 1e-9}

                    if not Ri and not Sj:
                        continue

                    # only log the first few active slots per iteration to keep logs compact
                    mu_slot = {j: mu[(j, mon, t)] for j in master.I}
                    if debug_slots_printed < self.debug_first_k_slots:
                        _log_slot_snapshot(master, it, mon, t, Ri, Sj, mu_slot=mu_slot)
                        debug_slots_printed += 1

                    # quick profitability check; if no arc is profitable, lock Θ=0 once and skip
                    if not hasattr(self, '_zero_theta_added'):
                        self._zero_theta_added = set()
                    
                    A = active_pairs_for_slot(mon, t)
                    profitable = any(
                        (i in Ri and j in Sj) and (pbar[j] - mu_slot[j] - T_TRIP.get((i, j), float('inf'))/x_kWh > 1e-9)
                        for (i, j) in A
                    )
                    if not profitable:
                        master.benders_cuts.add(master.ThetaRedir[mon, t] <= 0)
                        continue

                    # --- Dynamic safe UB for ThetaRedir(mon,t), add once per (mon,t) per iteration ---
                    if Ri and Sj:
                        if not hasattr(self, '_added_dyn_ub') or self._added_dyn_ub.get((mon, t)) != it:
                            A = active_pairs_for_slot(mon, t)
                            try:
                                min_trip_per_kwh = min(T_TRIP[(i, j)]/x_kWh for (i, j) in A if i in Ri and j in Sj)
                            except ValueError:
                                min_trip_per_kwh = 0.0
                            unit_ub = max(0.0, PRICE_REF - min(mu_slot.values()) - min_trip_per_kwh)
                            rhs_slot = N_MONTH[mon] * unit_ub * min(sum(Ri.values()), sum(Sj.values()))
                            master.benders_cuts.add(master.ThetaRedir[mon, t] <= rhs_slot)
                            if not hasattr(self, '_added_dyn_ub'):
                                self._added_dyn_ub = {}
                            self._added_dyn_ub[(mon, t)] = it

                    spLP = create_subproblem_lp (mon, t, Ri, Sj, mu_slot, pbar)
                    resS = optLP.solve(spLP)
                    termS = getattr(resS, 'termination_condition', getattr(resS, 'solver', None) and resS.solver.termination_condition)
                    if not _is_optimal_tc(termS):
                        raise RuntimeError(f"LP redirection SP ({mon},{t}) not optimal: {termS}")

                    # keep a smoothed core point
                    self._update_core_point(mon, t, Ri, Sj)
                    
                    # get duals
                    alpha, beta = extract_transport_duals(spLP)
                    self._update_core_point(mon, t, Ri, Sj)        # keep a smoothed core point
                    # Pareto-optimal rescaling at core point
                    alpha, beta = self._mw_rescale(mon, t, alpha, beta)  # MW normalization
                    
                    # ensure the dual suffix exists (it should, but this is cheap)
                    if not hasattr(spLP, 'dual'):
                        spLP.dual = Suffix(direction=Suffix.IMPORT)

                    has_supply   = hasattr(spLP, 'Supply')
                    has_capacity = hasattr(spLP, 'Capacity')
                    skipped_supply   = list(spLP.I) if not has_supply   else [i for i in spLP.I if i not in spLP.Supply]
                    skipped_capacity = list(spLP.J) if not has_capacity else [j for j in spLP.J if j not in spLP.Capacity]

                    if skipped_supply or skipped_capacity:
                        logger.debug(f"      skipped Supply idx={skipped_supply}  skipped Capacity idx={skipped_capacity}")
                    lp_obj_val = float(value(spLP.obj))
                    if lp_obj_val >= 5e4: 
                        slot_summ.append((lp_obj_val, mon, t, sum(Ri.values()), sum(Sj.values())))
                    
                    # --- NEW: fast integer rounding heuristic for LB (feasible w.r.t. R/S) ---
                    if hasattr(spLP, 'A'):
                        # Pull LP flows
                        z_lp = {}
                        for (i, j) in list(spLP.A):
                            try:
                                z_lp[i, j] = max(0.0, float(value(spLP.z[i, j])))
                            except Exception:
                                z_lp[i, j] = 0.0
                    
                        # Residual supply/capacity
                        r_rem = {i: Ri.get(i, 0.0) for i in Ri}
                        s_rem = {j: Sj.get(j, 0.0) for j in Sj}
                    
                        # Sort arcs by LP flow (desc), greedy allocate integer trips
                        for (i, j), z in sorted(z_lp.items(), key=lambda kv: kv[1], reverse=True):
                            if r_rem.get(i, 0.0) < x_kWh or s_rem.get(j, 0.0) < x_kWh:
                                continue
                            # Max integer trips respecting residuals
                            ntrip = min(int(z // x_kWh), int(r_rem[i] // x_kWh), int(s_rem[j] // x_kWh))
                            if ntrip <= 0:
                                continue
                            energy = ntrip * x_kWh
                    
                            # per-kWh profit consistent with LP coef
                            unit_profit = max(0.0, pbar[j] - mu_slot[j] - T_TRIP[(i, j)]/x_kWh)
                            theta_heur_total += N_MONTH[mon] * unit_profit * energy
                    
                            # update residuals
                            r_rem[i] -= energy
                            s_rem[j] -= energy

                    if debug_slots_printed <= self.debug_first_k_slots:
                        _log_slot_snapshot(master, it, mon, t, Ri, Sj, mu_slot=None, lp_obj=lp_obj_val)
                        _log_sp_top_flows(spLP)

                    has_arcs = hasattr(spLP, 'A') and len(list(spLP.A)) > 0
                    if has_arcs and (hasattr(spLP, 'Supply') or hasattr(spLP, 'Capacity')):
                        alpha, beta = extract_transport_duals(spLP)
                    else:
                        alpha, beta = {}, {}
                    
                    # Only add the cut if violated (Benders optimality cut)
                    theta_val = float(value(master.ThetaRedir[mon, t]))
                    # duals already reflect the scaling used in spLP; do NOT multiply by N_MONTH here
                    rhs_val = (sum(alpha.get(i, 0.0) * Ri.get(i, 0.0) for i in Ri)
                               + sum(beta.get(j, 0.0) * Sj.get(j, 0.0) for j in Sj))
                    key = (mon, t)
                    # add if violated OR if it tightens our best-known UB for this slot
                    if (rhs_val + 1e-6 < theta_val) or (rhs_val + 1e-6 < self.best_theta_rhs[key]):
                        _ = add_transport_cut(master, mon, t, alpha, beta)
                        self.best_theta_rhs[key] = min(self.best_theta_rhs[key], rhs_val)
                        added_cuts += 1
                        logger.debug(f"      +cut ({mon},{t}) RHS={rhs_val:.2f}  Θ={theta_val:.2f}  bestRHS={self.best_theta_rhs[key]:.2f}")

                    # Optional MIP SP for exact integer recourse every eval_mip_every iterations
                    if (it % self.eval_mip_every) == 0:
                        # quick check: any arc with positive unit margin under current μ
                        A = active_pairs_for_slot(mon, t)
                        profitable = any(
                            (i in Ri and j in Sj) and (pbar[j] - mu_slot[j] - T_TRIP.get((i, j), float('inf'))/x_kWh > 1e-9)
                            for (i, j) in A
                        )
                        if profitable:
                            spMIP = create_subproblem_mip(mon, t, Ri, Sj, mu_slot, pbar, enforce_min_quantum=False)
                            resMi = optSM.solve(spMIP)
                            termMi = getattr(resMi, 'termination_condition',
                                             getattr(resMi.solver, 'termination_condition', None))
                            if _is_optimal_tc(termMi):
                                mip_val = float(value(spMIP.obj))   # θ* for this (mon,t) at current R,S,μ,prices
                                theta_mip_total += mip_val
                                # --- add integer (logic-based) optimality cut on θ_{mon,t}
                                add_integer_optimality_cut(master, mon, t, mip_val, Ri, Sj, mu_slot, pbar)
                                if debug_slots_printed <= self.debug_first_k_slots:
                                    _log_slot_snapshot(master, it, mon, t, Ri, Sj, mu_slot=None, mip_obj=mip_val)

            # Build LB from (master current obj – master’s ThetaRedir) + MIP’d ThetaRedir
            theta_master_total = sum(value(master.ThetaRedir[mon, t]) for mon in master.M for t in master.H)
            # Prefer exact MIP θ if available; otherwise use heuristic θ
            theta_feas = theta_mip_total if (it % self.eval_mip_every) == 0 and theta_mip_total > 0 else theta_heur_total
            LB_cand = value(master.obj) - theta_master_total + max(0.0, theta_feas)
            
            top = sorted(slot_summ, key=lambda x: x[0], reverse=True)[:5]
            logger.info(
                f"[it {it:02d}] cuts={len(master.benders_cuts):5d} "
                f"Theta(master)={theta_master_total:,.0f} "
                f"{'(MIP '+str(theta_mip_total)+')' if (it % self.eval_mip_every)==0 else ''} "
                f"UB={best_UB:,.0f} LB={best_LB:,.0f} gap={(best_UB-best_LB)/max(1,abs(best_UB),abs(best_LB))*100:5.2f}%"
            )
            for lp, mon, t, rtot, stot in top:
                logger.debug(f"   top slot {mon:>9},{t:02d}: R={rtot:6.2f} S={stot:6.2f} LP_obj={lp:10,.2f}")

            if theta_mip_total > theta_master_total + 1e-6:
                logger.warning(
                    "theta_mip_total (actual) > master Theta sum. Initial Theta UBs are too tight; "
                    "increase UNIT_VALUE_UB or recheck bounds."
                )

            if LB_cand > best_LB + 1e-9:
                best_LB = LB_cand
                best_x = dict(x_sol)
                best_PV = dict(pv_sol)
                best_Batt = dict(batt_sol)

            if best_LB > best_UB + 1e-6:
                logger.warning("LB > UB detected; most likely initial Theta bounds were too tight this round. "
                               "This will resolve once Θ upper bounds are non-binding.")
            # Keep UB nonincreasing and LB nondecreasing; clip reported gap at 0
            best_UB = min(best_UB, UB_cand)
            best_LB = max(best_LB, LB_cand)
            gap = max(0.0, (best_UB - best_LB) / max(1.0, abs(best_UB), abs(best_LB)))

            self.iteration_history.append({
                'iteration': it,
                'cuts_added': added_cuts,
                'upper_bound': best_UB,
                'lower_bound': best_LB,
                'gap': gap
            })
            logger.info(f"Iter {it:02d}: UB={best_UB:,.0f}  LB={best_LB:,.0f}  gap={100*gap:.2f}%  +cuts={added_cuts}")
                        
            # ---------- Early-stop rules ----------
            # (A) stop immediately if no new cuts were added this round
            if self.stop_if_no_cuts and added_cuts == 0:
                logger.info("Early stop: no Benders cuts added this iteration.")
                break

            # (B) stop if no progress for more than `no_progress_patience` iterations
            prev = self._prev_metrics
            if prev['gap'] is not None:
                # Relative improvements (cap denominators to avoid div by 0)
                rel_gap_impr = (prev['gap'] - gap) / max(1.0, prev['gap'])
                rel_ub_impr  = (prev['UB'] - best_UB) / max(1.0, abs(prev['UB'])) if prev['UB'] is not None else 0.0
                rel_lb_impr  = (best_LB - prev['LB']) / max(1.0, abs(prev['LB'])) if prev['LB'] is not None else 0.0
                if max(rel_gap_impr, rel_ub_impr, rel_lb_impr) <= self.rel_improve_tol:
                    self._stall_count += 1
                else:
                    self._stall_count = 0

                if self._stall_count > self.no_progress_patience:
                    logger.info(f"Early stop: no meaningful progress for {self._stall_count} iterations "
                                f"(tol={self.rel_improve_tol}).")
                    break

            # remember metrics for next round
            self._prev_metrics = {'UB': best_UB, 'LB': best_LB, 'gap': gap}

        # APPSI doesn’t require manual license release, but keep it safe
        try:
            optM.release_license()
        except Exception: pass
            
        # --- Final certification pass (exact recourse at best design) ---
        redir_kwh_total   = 0.0
        redir_trips_total = 0
        redir_rev_total   = 0.0
        redir_cost_total  = 0.0
        
        if best_x is not None:
            # 1) Fix the *design* on a clone
            m_best = master.clone()
            for (i, c), v in best_x.items():
                m_best.x[i, c].fix(int(round(v)))
            for i, v in (best_PV or {}).items():
                m_best.PV[i].fix(int(round(v)))
            for i, v in (best_Batt or {}).items():
                m_best.Batt[i].fix(int(round(v)))
        
            # 2) Build a *relaxed* LP copy to get duals (relaxes all integers, incl. delta)
            m_best_lp = m_best.clone()
            TransformationFactory('core.relax_integer_vars').apply_to(m_best_lp)

            # Price mix under the fixed best design (for certification MIP recourse)
            pbar_best = _price_bar_by_cell_from_design(m_best_lp)
        
            if not hasattr(m_best_lp, 'dual'):
                m_best_lp.dual = Suffix(direction=Suffix.IMPORT)
        
            res_best = self._configure_lp_solver().solve(m_best_lp)
            term_best = getattr(res_best, 'termination_condition',
                                getattr(res_best.solver, 'termination_condition', None))
            if not _is_optimal_tc(term_best):
                raise RuntimeError(f"Operational LP (best design) not optimal: {term_best}")
        
            # Safe dual getter (missing duals -> 0.0)
            def _safe_dual(m, con):
                try:
                    return float(m.dual[con])
                except KeyError:
                    return 0.0
        
            # μ from the relaxed LP
            mu_best = {
                (j, mon, t): _safe_dual(m_best_lp, m_best_lp.EnergyBalance[j, mon, t])
                for j in m_best_lp.I for mon in m_best_lp.M for t in m_best_lp.H
            }
        
            # 3) Exact recourse by MIP SP with R,S from the LP and μ from above
            theta_mip_final = 0.0
            optSM = self._configure_mip_solver()
                        
            for mon in MONTHS:
                for t in INTERVALS:
                    Ri = {i: float(value(m_best_lp.R[i, mon, t]))
                          for i in m_best_lp.I if value(m_best_lp.R[i, mon, t]) > 1e-9}
                    Sj = {j: float(value(m_best_lp.S[j, mon, t]))
                          for j in m_best_lp.I if value(m_best_lp.S[j, mon, t]) > 1e-9}
                    if not Ri or not Sj:
                        continue
        
                    mu_slot = {j: mu_best[(j, mon, t)] for j in m_best_lp.I}
                    spMIP = create_subproblem_mip(mon, t, Ri, Sj, mu_slot, pbar_best, enforce_min_quantum=False)
                    resMi = optSM.solve(spMIP)
                    termMi = getattr(resMi, 'termination_condition',
                                     getattr(resMi.solver, 'termination_condition', None))
                    if _is_optimal_tc(termMi):
                        mip_val = float(value(spMIP.obj))   # θ* for this (mon,t) at current R,S,μ,prices
                        theta_mip_final += mip_val
                        # (Optional) adding a cut here won't change the final certificate, but harmless if kept.
                        add_integer_optimality_cut(master, mon, t, mip_val, Ri, Sj, mu_slot, pbar_best)
        
                        if hasattr(spMIP, 'A'):
                            for (i, j) in list(spMIP.A):
                                z_ij = float(value(spMIP.z[i, j]))           # kWh in that slot
                                n_ij = int(round(value(spMIP.ntrip[i, j])))  # trips in that slot
                                ndays = N_MONTH[mon]
        
                                # Annualize energy and revenues/costs
                                redir_kwh_total   += ndays * z_ij
                                redir_trips_total += n_ij
                                redir_rev_total   += ndays * pbar_best[j] * z_ij
                                redir_cost_total  += ndays * T_TRIP[(i, j)] * n_ij
                                
            # 4) Replace LP Θ by exact MIP θ in the certified LB
            theta_master_total = sum(float(value(m_best_lp.ThetaRedir[mm, tt]))
                                     for mm in m_best_lp.M for tt in m_best_lp.H)
            certified_LB = float(value(m_best_lp.obj)) - theta_master_total + theta_mip_final
            best_LB = min(best_UB, max(best_LB, certified_LB))
            logger.info(f"Actual LB (MIP recourse at best design): {certified_LB:,.0f} SEK")
        
            # 5) Build ops summary **from the solved LP** (m_best_lp)
            def _ann_sum_lp(v):
                return sum(N_MONTH[m] * float(value(v[i, m, t]))
                           for i in m_best_lp.I for m in m_best_lp.M for t in m_best_lp.H)
        
            ops = {
                'pv_used_direct_kwh'   : _ann_sum_lp(m_best_lp.pv_dir),
                'pv_to_batt_kwh'       : _ann_sum_lp(m_best_lp.pv_batt),
                'grid_used_direct_kwh' : _ann_sum_lp(m_best_lp.grid_dir),
                'grid_to_batt_kwh'     : _ann_sum_lp(m_best_lp.grid_batt),
                'battery_discharge_kwh': _ann_sum_lp(m_best_lp.batt_discharge),
                'unmet_home_kwh'       : sum(N_MONTH[m]*float(value(m_best_lp.slack[i, m, t, 'home']))
                                             for i in m_best_lp.I for m in m_best_lp.M for t in m_best_lp.H),
                'unmet_public_kwh'     : sum(N_MONTH[m]*float(value(m_best_lp.slack[i, m, t, 'public']))
                                             for i in m_best_lp.I for m in m_best_lp.M for t in m_best_lp.H),
                'redir_energy_kwh'     : redir_kwh_total,
                'redir_trips'          : redir_trips_total,
                'redir_revenue_sek'    : redir_rev_total,
                'redir_cost_sek'       : redir_cost_total,
            }
            ops['unmet_total_kwh'] = ops['unmet_home_kwh'] + ops['unmet_public_kwh']
        
        else:
            # No incumbent design captured — return zeroed ops
            ops = {
                'pv_used_direct_kwh':0.0,'pv_to_batt_kwh':0.0,'grid_used_direct_kwh':0.0,
                'grid_to_batt_kwh':0.0,'battery_discharge_kwh':0.0,'unmet_home_kwh':0.0,
                'unmet_public_kwh':0.0,'unmet_total_kwh':0.0,'redir_energy_kwh':0.0,
                'redir_trips':0,'redir_revenue_sek':0.0,'redir_cost_sek':0.0
            }


        return {
            'x': best_x,
            'PV': best_PV,
            'Batt': best_Batt,
            'upper_bound': best_UB,
            'lower_bound': best_LB,
            'iterations': self.iteration_history,
            'converged': (best_UB - best_LB) <= self.tolerance * max(1.0, abs(best_UB), abs(best_LB)),
            'ops': ops,
            'master': master
        }

In [8]:
def main():
    logger.info("Starting LBBD Optimization")
    logger.info(f"Problem size: {len(HexIDs)} cells × {len(MONTHS)} months × {len(INTERVALS)} half-hours")
    logger.info(f"Data loaded: {len(HexIDs)} cells, {len(Demand_event_annual)} demand entries")
    logger.info(f"Tariffs [SEK/kWh]  slow={charger_price['slow']:.2f}  medium={charger_price['medium']:.2f}  fast={charger_price['fast']:.2f}")
    logger.info(f"Daily annuity [SEK/day]  slow={daily_cost['slow']:.1f}  medium={daily_cost['medium']:.1f}  fast={daily_cost['fast']:.1f}  PV={daily_cost['PV']:.1f}  Batt={daily_cost['Batt']:.1f}")
    logger.info(f"Redirection: radius={MAX_DIST} km  |allowed pairs|={len(allowed)}  |arc-slots|={len(ALLOWED_ST)}  x_kWh={x_kWh}  redir_min={redir_min}")
    logger.info(f"Active arc-slots |A| = {len(ALLOWED_ST)} (distance-filtered and demand-active)")

    start_time = time.time()

    # Tighter tolerance is fine because SP is cheap (transport LP), but keep iterations capped.
    lbbd = LogicBasedBenders(tolerance=0.001, max_iterations=25, eval_mip_every=1, submip_timelimit=None, debug_first_k_slots=0)

    try:
        results = lbbd.solve()
    except Exception as e:
        logger.error(f"LBBD solve failed: {e}", exc_info=True)
        return None

    end_time = time.time()
    logger.info(f"LBBD completed in {end_time - start_time:.2f} seconds")

    if not results:
        logger.warning("No results dictionary returned.")
        return None

    # --------------- Report results ---------------
    x_sol    = results.get('x', {}) or {}
    pv_sol   = results.get('PV', {}) or {}
    batt_sol = results.get('Batt', {}) or {}

    # Charger totals (per type) and per-cell installations
    total_chargers = {c: 0 for c in C_PUB}
    install_map = defaultdict(lambda: {'slow':0,'medium':0,'fast':0})

    for (i, c), qty in x_sol.items():
        q = int(round(qty))
        if q > 0:
            total_chargers[c] += q
            install_map[i][c] += q

    tot_pv   = int(sum(int(round(v)) for v in pv_sol.values()))
    tot_batt = int(sum(int(round(v)) for v in batt_sol.values()))

    logger.info("="*64)
    logger.info("OPTIMIZATION RESULTS SUMMARY")
    logger.info("="*64)

    # Installations summary
    logger.info("CHARGER INSTALLATION SUMMARY:")
    logger.info("-"*40)
    for c in C_PUB:
        logger.info(f"  {c.upper():6} chargers: {total_chargers[c]:4d} units")
    logger.info(f"  {'TOTAL':6} chargers: {sum(total_chargers.values()):4d} units")

    logger.info("")
    logger.info("ON-SITE GENERATION & STORAGE:")
    logger.info("-"*40)
    logger.info(f"  PV panels      : {tot_pv:6d} units")
    logger.info(f"  Battery cells  : {tot_batt:6d} units")

    # Annual CapEx from daily annuities (consistent with monolithic economics)
    capex_ch = DAYS * sum(daily_cost[c] * total_chargers[c] for c in C_PUB)
    capex_pv = DAYS * daily_cost['PV']   * tot_pv
    capex_bt = DAYS * daily_cost['Batt'] * tot_batt
    capex_all = capex_ch + capex_pv + capex_bt

    logger.info("")
    logger.info("FINANCIAL SUMMARY:")
    logger.info("-"*40)
    logger.info(f"  Annual CapEx (chargers): {capex_ch:,.0f} SEK")
    logger.info(f"  Annual CapEx (PV)      : {capex_pv:,.0f} SEK")
    logger.info(f"  Annual CapEx (Battery) : {capex_bt:,.0f} SEK")
    logger.info(f"  Annual CapEx (total)   : {capex_all:,.0f} SEK")

    ub = results.get('upper_bound', float('nan'))
    lb = results.get('lower_bound', float('nan'))
    gap = (ub - lb) / max(1.0, abs(ub), abs(lb)) if (ub is not None and lb is not None) else float('nan')
    uh = results.get('unmet_home_kwh', 0.0)
    up = results.get('unmet_public_kwh', 0.0)
    ut = results.get('unmet_total_kwh', 0.0)
    logger.info("")
    logger.info(f"  Lower Bound (Profit)   : {lb:,.0f} SEK")
    logger.info(f"  Upper Bound (Profit)   : {ub:,.0f} SEK")
    logger.info(f"  Final MIP Gap          : {100.0*gap:.3f}%")

    if results.get('converged', False):
        logger.info("  Status: CONVERGED")
    else:
        logger.info("  Status: NOT CONVERGED")

    # # Top-10 cells by total public chargers
    # logger.info("")
    # logger.info("TOP-10 CELLS BY PUBLIC CHARGER INSTALLATIONS:")
    # logger.info("-"*48)
    # ranked = sorted(install_map.items(), key=lambda kv: sum(kv[1].values()), reverse=True)[:10]
    # for i, d in ranked:
    #     total_i = sum(d.values())
    #     if total_i > 0:
    #         logger.info(f"  HexID {i:>5}: slow={d['slow']:3d}, medium={d['medium']:3d}, fast={d['fast']:3d}  (sum={total_i})")

    # ---------- Operational summary ----------
    ops = results.get('ops', {}) or {}
    logger.info("")
    logger.info("==================  BREAKDOWN  =================")
    # Revenue from local (public+home) already sits in the master objective via edisp;
    # If you want to show it explicitly, recompute from the returned 'master' model.
    master = results.get('master', None)
    if master is not None:
        rev_local = sum(N_MONTH[m]*float(value(master.Price[c]))*float(value(master.edisp[i,m,t,c,b]))
                        for i in master.I for m in master.M for t in master.H for b in master.B for c in master.C)
        grid_cost = (
            sum(N_MONTH[m]*float(value(master.tou[m,t]))*float(value(master.grid_dir [i,m,t]))
                for i in master.I for m in master.M for t in master.H) +
            sum(N_MONTH[m]*float(value(master.tou[m,t]))*float(value(master.grid_batt[i,m,t]))
                for i in master.I for m in master.M for t in master.H)
        )
        redir_rev  = ops.get('redir_revenue_sek', 0.0)
        redir_cost = ops.get('redir_cost_sek', 0.0)
        slack_pen  = 0.0  # you can compute from 'master.slack' similarly if needed

        logger.info(f"Revenue (all chargers)        : {rev_local:>13,.0f}")
        logger.info(f"  – share from redirected cars : {redir_rev:>13,.0f}")
        logger.info(f"Opex    – grid purchases      : {grid_cost:>13,.0f}")
        logger.info(f"Opex    – redirection cost    : {redir_cost:>13,.0f}")
        logger.info(f"Opex    – unmet-demand pen.   : {slack_pen:>13,.0f}")
    logger.info("")
    logger.info("OPERATIONAL SUMMARY (annualized):")
    logger.info("-"*40)
    logger.info(f"  Energy redirected      : {ops.get('redir_energy_kwh', 0.0):,.1f} kWh")
    if ops.get('redir_trips', 0) > 0:
        logger.info(f"  Redirected trips       : {ops['redir_trips']:,.0f}")
    logger.info(f"  PV used directly       : {ops.get('pv_used_direct_kwh', 0.0):,.1f} kWh")
    logger.info(f"  PV -> battery          : {ops.get('pv_to_batt_kwh', 0.0):,.1f} kWh")
    logger.info(f"  Grid used directly     : {ops.get('grid_used_direct_kwh', 0.0):,.1f} kWh")
    logger.info(f"  Grid -> battery        : {ops.get('grid_to_batt_kwh', 0.0):,.1f} kWh")
    logger.info(f"  Battery discharge      : {ops.get('battery_discharge_kwh', 0.0):,.1f} kWh")
    logger.info("-"*40)

    if not ops: logger.warning("Operational summary printed with zeroed values because evaluate_ops() is not available.")
    return results

if __name__ == "__main__":
    _ = main()

2025-10-09 18:18:49,127 - __main__ - INFO - Starting LBBD Optimization
2025-10-09 18:18:49,129 - __main__ - INFO - Problem size: 63 cells × 12 months × 48 half-hours
2025-10-09 18:18:49,131 - __main__ - INFO - Data loaded: 63 cells, 72576 demand entries
2025-10-09 18:18:49,133 - __main__ - INFO - Tariffs [SEK/kWh]  slow=5.50  medium=6.00  fast=6.50
2025-10-09 18:18:49,135 - __main__ - INFO - Daily annuity [SEK/day]  slow=3.2  medium=6.1  fast=81.9  PV=1.0  Batt=11.3
2025-10-09 18:18:49,136 - __main__ - INFO - Redirection: radius=1.5 km  |allowed pairs|=57  |arc-slots|=20376  x_kWh=20.0  redir_min=0.1
2025-10-09 18:18:49,138 - __main__ - INFO - Active arc-slots |A| = 20376 (distance-filtered and demand-active)
2025-10-09 18:23:01,400 - __main__ - INFO - [it 01] Master snapshot: sum_x=229  PV=7416  Batt=100
2025-10-09 18:25:50,864 - __main__ - INFO - [it 01] cuts=  672 Theta(master)=42,389,876 (MIP 2153.43239813504) UB=77,523,398 LB=-inf gap=  nan%
2025-10-09 18:25:50,866 - __main__ - IN